# TD Kubernetes

#### On va créér un namespace `dockercoins`, ce sera plus propre pour gérer nos ressources

In [ ]:
!kubectl create namespace dockercoins
!kubectl config set-context --current --namespace=dockercoins

In [ ]:
!kubectl get namespaces

## I. Déployer l'application Dockercoins avec un fonctionnement impératif

### 1. Déployer les containers

#### Déployer un container ```redis```

In [ ]:
!kubectl create deployment redis --image redis

#### Déployer un container ```hasher```

In [ ]:
!kubectl create deployment hasher --image dockercoins/hasher:v0.1

In [ ]:
!kubectl get pods,replicaset,deployment

#### Déployer un container ```rng```

In [ ]:
!kubectl create deployment rng --image dockercoins/rng:v0.1

#### Déployer un container ```worker```

In [ ]:
!kubectl create deployment worker --image dockercoins/worker:v0.1

#### Déployer un container ```webui```

In [ ]:
!kubectl create deployment webui --image dockercoins/webui:v0.1

#### Checker l'état des pods, replicaset et deployment

In [ ]:
!kubectl get pods,replicaset,deployment -n dockercoins

#### Tenter de vous connecter sur la webui

http://localhost:8000/index.html

Ça ne marche pas, et c'est normal : aucun pods n'est accessible depuis le localhost

In [ ]:
!kubectl logs deployment.apps/worker

#### Aie, on voit qu'il ne connait pas redis : "Error -3 connecting to redis:6379. Try again."

-> il faut exposer les ports des services pour qu'ils communiquent entre eux !

### 2. Exposer les services

#### Regarder les services et leurs IPs

In [ ]:
!kubectl get services

#### Exposer le service ```redis```

In [ ]:
!kubectl expose deployment redis --port 6379

In [ ]:
!kubectl get services -n dockercoins

Le service `redis` a bien une `CLUSTER_IP`

In [ ]:
!kubectl expose deployment rng --port 80
!kubectl expose deployment hasher --port 80
!kubectl expose deployment webui --port 80 --type LoadBalancer

In [ ]:
!kubectl get pods -n dockercoins -o wide

In [ ]:
!kubectl get services -n dockercoins

#### Forward de port pour acceder à la webui

In [ ]:
# cellule à stoper ensuite pour pouvoir continuer le TD
!kubectl port-forward service/webui 1234:80

(On aurait pu passer par un LoadBalancer ou Node Port)

#### Tenter de vous connecter sur la webui

http://localhost:1234/index.html

#### Clean de toutes les ressources créées dans notre namespace `dockercoins`

In [ ]:
!kubectl delete services,deployments --all -n dockercoins

## II. Déployer l'application Dockercoins avec un fonctionnement déclaratif

Kubernetes exécute les conteneurs pour vous, donc au lieu de les exécuter explicitement avec docker container run, etc. vous décrivez le résultat souhaité dans un fichier `YAML` et lorsque vous exécutez `kubectl apply`, Kubernetes démarre tous les conteneurs.

Pour se faire, on va passer par un fichier `dockercoins.yaml` :

In [ ]:
!kubectl create deployment hasher  -o yaml --dry-run --image dockercoins/hasher:v0.1

In [ ]:
!cat dockercoins.yaml

In [ ]:
!kubectl apply -f dockercoins.yaml

#### Vérification que tout a bien été créés

In [ ]:
!kubectl get pods,replicaset,deployment

In [ ]:
!kubectl get services -n dockercoins

#### Forward de port pour acceder à la webui

In [ ]:
# cellule à stoper ensuite pour pouvoir continuer le TD
!kubectl port-forward service/webui 1234:80

http://localhost:1234/index.html

#### Technique du dry-run (en réalité ça ne va rien créér) pour avoir une conf yaml à peu près réutilisable dans un ```kubectl apply``` :

In [ ]:
!kubectl create deployment rng --image dockercoins/rng:v01 -o yaml --dry-run

## III. Scalabilité

L'idée est maintenant de passer à l'échelle et de scaler le pod worker !

In [ ]:
!kubectl get pods,replicaset,deployment,service

#### Faites un scale up

In [ ]:
!kubectl scale deployment worker --replicas 2

#### Que remarquez-vous ?

In [ ]:
!kubectl get pods,replicaset,deployment

#### Lancer cette commande dans un terminal :
```shell
kubectl port-forward service/webui 1234:80
```
et aller de nouveau sur la webui : http://localhost:1234/index.html
Que remarquez-vous ?

#### Faite un scale down. Que remarquez-vous ?

In [ ]:
!kubectl scale deployment worker --replicas 1

In [ ]:
!kubectl get pods -o wide

#### Faite un scale up avec trois replicas. Que remarquez-vous ?

In [ ]:
!kubectl scale deployment worker --replicas 3

In [ ]:
!kubectl get pods -o wide

#### Clean de toutes les ressources créées dans notre namespace `dockercoins`

In [1]:
!kubectl delete replicasets,services,deployments,pods --all -n dockercoins

#### On détruit le namespace `dockercoins`

In [ ]:
!kubectl delete namespace dockercoins
!kubectl config set-context --current --namespace=default